In [1]:
import cv2
import numpy as np
import cvzone
import math
import mediapipe as mp
import time
caminput =0

In [2]:
##mediapipe hand detection
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose


<img src="https://i.imgur.com/qpRACer.png "/>

<img src="https://camo.githubusercontent.com/7fbec98ddbc1dc4186852d1c29487efd7b1eb820c8b6ef34e113fcde40746be2/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f706f73655f747261636b696e675f66756c6c5f626f64795f6c616e646d61726b732e706e67">

In [3]:
cap = cv2.VideoCapture(caminput)
StartCoords = []
pose = mp.solutions.pose.Pose(model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5)
hand = mp.solutions.hands.Hands(max_num_hands=2,
    min_detection_confidence=0.5)
prev_time = 0
new_time = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image = np.zeros(frame.shape, dtype=np.uint8)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        img.flags.writeable = False
        res = holistic.process(img)
        pose_res = pose.process(img)
        hand_res = hand.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(img, res.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(255, 110, 10), thickness=1, circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(255, 256, 121), thickness=1, circle_radius=1)
                                    )
        # if hand_res.multi_hand_landmarks:
        #     for hand_landmarks in hand_res.multi_hand_landmarks:
        #         mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(img, pose_res.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                            )
        if pose_res.pose_landmarks:
            left_hand_end_point = np.array([pose_res.pose_landmarks.landmark[14].x, pose_res.pose_landmarks.landmark[14].y, pose_res.pose_landmarks.landmark[14].z])
            left_hand_start_point = np.array([pose_res.pose_landmarks.landmark[16].x, pose_res.pose_landmarks.landmark[16].y, pose_res.pose_landmarks.landmark[16].z])
            por = 1/9
            Vect_left_hand = (left_hand_end_point - left_hand_start_point)*por + left_hand_start_point
            cv2.circle(img, (int(Vect_left_hand[0]*img.shape[1]), int(Vect_left_hand[1]*img.shape[0])), 5, (0, 0, 255), -1)
            right_hand_end_point = np.array([pose_res.pose_landmarks.landmark[13].x, pose_res.pose_landmarks.landmark[13].y, pose_res.pose_landmarks.landmark[13].z])
            right_hand_start_point = np.array([pose_res.pose_landmarks.landmark[15].x, pose_res.pose_landmarks.landmark[15].y, pose_res.pose_landmarks.landmark[15].z])
            Vect_right_hand = (right_hand_end_point - right_hand_start_point)*por + right_hand_start_point
            cv2.circle(img, (int(Vect_right_hand[0]*img.shape[1]), int(Vect_right_hand[1]*img.shape[0])), 5, (0, 0, 255), -1)
        new_time = time.time()
        fps = 1/(new_time-prev_time)
        prev_time = new_time
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)
        cv2.imshow("pose detection", image)

        cv2.imshow("Hand Tracking", img)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()